In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup



Historical season betting data

In [13]:
hist_odds = pd.read_excel("nba odds 2020-21.xlsx")
hist_vist = hist_odds.iloc[::2]
hist_vist.index = range(533) 
hist_vist.loc[:, 'VH'] = 0
hist_vist['Team_V'] = hist_vist.loc[:, 'Team']
hist_vist['Final_V'] = hist_vist.loc[:,'Final']
hist_vist['O/U'] = hist_vist.loc[:,'Open']
hist_vist['ML_V'] = hist_vist.loc[:,'ML']
hist_vist = hist_vist[['VH', 'Team_V', 'Final_V', 'O/U', 'ML_V']]

hist_home = hist_odds.iloc[1::2]
hist_home.index = range(533) 
hist_home.loc[:, 'VH'] = 1
hist_home['Team_H'] = hist_home.loc[:, 'Team']
hist_home['Final_H'] = hist_home.loc[:,'Final']
hist_home['line'] = hist_home.loc[:, 'Open']
hist_home['ML_H'] = hist_home.loc[:, 'ML']
hist_home = hist_home[['VH', 'Team_H', 'Final_H', 'line', 'ML_H']]

hist_odds = pd.merge(left = hist_vist, right = hist_home, left_index = True, right_index = True, how = "outer")
hist_odds = hist_odds.replace('pk', 0)
hold1 = hist_odds['O/U']
hold2 = hist_odds['line']
hist_odds['O/U'] = [i if abs(i) > abs(j) else j for i,j in zip(hold1,hold2)]
hist_odds['line'] = [i if abs(i) < abs(j) else j for i,j in zip(hold1,hold2)]

line_sign = [-1 if x >= 0 else 1 for x in hist_odds['ML_H']]
hist_odds['line'] = hist_odds['line'] * line_sign
hist_odds.to_csv("hist_odds.csv")

C:\Users\thyang\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\thyang\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\thyang\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

Today's lines

In [12]:
URL = 'https://www.scoresandodds.com/nba'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')
ml = soup.find_all('span', class_ = 'data-moneyline')
money_line = [int(line.text) for line in ml]
team_name = soup.find_all('span', class_ = 'team-name')
teams = [name.text[0:(len(name.text)-5)] for name in team_name]
today_lines = {'teams': teams, 'ml' : money_line}
today_lines = pd.DataFrame(today_lines)
today_lines.to_csv("today_lines.csv")

Bet Sizing

In [ ]:
init_money = 100
prob_dif = pred_prob - bet_prob
for prob_dif > .3:
    kelly = pred_prob - (1-pred_prob)/bet_prob
    bet = kelly * init_money
    bet_win = (1+ bet_prob) * outcome * bet
    init_money = bet_win + bet

